## Importing necessary libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import requests
from bs4 import BeautifulSoup as bs


In [277]:
pd.set_option("display.max_columns", 100)

In [278]:
df = pd.read_csv("coaster_db.csv")

In [279]:
df.shape

(1087, 56)

There are 1087 rows and 56 columns in the dataset

## Understanding Data

In [280]:
df.sample(5)

,coaster_name,Length,Speed,Location,Status,Opening date,Type,Manufacturer,Height restriction,Model,Height,Inversions,Lift/launch system,Cost,Trains,Park section,Duration,Capacity,G-force,Designer,Max vertical angle,Drop,Soft opening date,Fast Lane available,Replaced,Track layout,Fastrack available,Soft opening date.1,Closing date,Opened,Replaced by,Website,Flash Pass Available,Must transfer from wheelchair,Theme,Single rider line available,Restraint Style,Flash Pass available,Acceleration,Restraints,Name,year_introduced,latitude,longitude,Type_Main,opening_date_clean,speed1,speed2,speed1_value,speed1_unit,speed_mph,height_value,height_unit,height_ft,Inversions_clean,Gforce_clean
524,Flashback (Six Flags New England),935 ft (285 m),47 mph (76 km/h),Other,NaN,NaN,Steel – Shuttle,Vekoma,48 in (122 cm),Boomerang,117 ft (36 m),3.0,Chain,NaN,NaN,North End,1:48,760 riders per hour,5.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single rider line available,NaN,Flash Pass available,NaN,NaN,NaN,2000,NaN,NaN,Steel,NaN,47 mph,76 km/h,47.0,mph,47.0,117.0,ft,NaN,3,5.2
811,Fiorano GT Challenge,"3,543 ft (1,079.9 m)",59 mph (95.0 km/h),Other,Operating,"November 4, 2010",Steel – Launched – Dueling,Maurer AG,51 in (130 cm),NaN,NaN,0.0,NaN,NaN,4 trains with 3 cars. Riders are arranged 2 ac...,NaN,NaN,NaN,1.5,Jack Rouse Associates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,24.4864,54.6042,Steel,2010-11-04,59 mph,95.0 km/h,59.0,mph,59.0,NaN,NaN,NaN,0,1.5
997,The Joker (Six Flags New England),"1,019 ft (311 m)",38 mph (61 km/h),Other,NaN,NaN,Steel – 4th Dimension – Wing Coaster,S&S – Sansei Technologies,48 in (122 cm),4D Free Spin,120 ft (37 m),6.0,Chain lift hill,NaN,5 trains with a single car. Riders are arrange...,NaN,NaN,720 riders per hour,NaN,Alan Schilke,NaN,54 ft (16 m),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single rider line available,NaN,Flash Pass available,NaN,NaN,NaN,2017,NaN,NaN,Steel,NaN,38 mph,61 km/h,38.0,mph,38.0,120.0,ft,NaN,6,NaN
378,The Great Nor'easter,"2,170 ft (660 m)",55 mph (89 km/h),Morey's Piers,Operating,"May 26, 1995",Steel – Inverted,Vekoma,52 in (132 cm),Suspended Looping Coaster-Custom,115 ft (35 m),5.0,Chain lift hill,"$5,500,000 USD",2 trains with 10 cars. Riders are arranged 2 a...,Surfside Pier,2:05,1200 riders per hour,NaN,NaN,NaN,95 ft (29 m),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1995,38.9893,-74.8023,Steel,1995-05-26,55 mph,89 km/h,55.0,mph,55.0,115.0,ft,NaN,5,NaN
739,Mystery Mine,"1,804.5 ft (550.0 m)",43.5 mph (70.0 km/h),Dollywood,Operating,"April 13, 2007",Steel – Euro-Fighter,Gerstlauer,48–76 in (122–193 cm),Euro-Fighter (Custom),85.3 ft (26.0 m),2.0,Two Chain lift hills,$17.5 million,7 trains with a single car. Riders are arrange...,Timber Canyon,2:30,1000 riders per hour,NaN,Werner Stengel,95°,NaN,"March 31, 2007",NaN,NaN,Terrain,NaN,"March 31, 2007",NaN,NaN,NaN,NaN,NaN,Must transfer from wheelchair,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007,35.7956,-83.5301,Steel,2007-04-13,43.5 mph,70.0 km/h,43.5,mph,43.5,85.3,ft,NaN,2,NaN


We could see that there are number of columns where there a 80% null values

In [281]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1087 entries, 0 to 1086
Data columns (total 56 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   coaster_name                   1087 non-null   object 
 1   Length                         953 non-null    object 
 2   Speed                          937 non-null    object 
 3   Location                       1087 non-null   object 
 4   Status                         874 non-null    object 
 5   Opening date                   837 non-null    object 
 6   Type                           1087 non-null   object 
 7   Manufacturer                   1028 non-null   object 
 8   Height restriction             831 non-null    object 
 9   Model                          744 non-null    object 
 10  Height                         965 non-null    object 
 11  Inversions                     932 non-null    float64
 12  Lift/launch system             795 non-null    o

#### Lets get the % of not nulls for each column

In [282]:
valid_cols = np.round((((~df.isna()).sum() * 100 )/ len(df)).sort_values(ascending=False))
nulls_df = valid_cols.reset_index(name="not_null_percent")
nulls_df

,index,not_null_percent
0,coaster_name,100.0
1,Location,100.0
2,Inversions_clean,100.0
3,year_introduced,100.0
4,Type,100.0
5,Type_Main,100.0
6,Manufacturer,95.0
7,Height,89.0
8,height_unit,89.0
9,height_value,89.0


### Dropping cols that have more than 80% null values

In [283]:
nulls_df = nulls_df[nulls_df['not_null_percent'] < 20]
nulls_df = nulls_df.rename(columns={"index": "col_name"})
cols = list(nulls_df['col_name'])

### Dropping 18 columns as they have more than 80% null values

In [284]:
df = df.drop(columns=cols, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1087 entries, 0 to 1086
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   coaster_name        1087 non-null   object 
 1   Length              953 non-null    object 
 2   Speed               937 non-null    object 
 3   Location            1087 non-null   object 
 4   Status              874 non-null    object 
 5   Opening date        837 non-null    object 
 6   Type                1087 non-null   object 
 7   Manufacturer        1028 non-null   object 
 8   Height restriction  831 non-null    object 
 9   Model               744 non-null    object 
 10  Height              965 non-null    object 
 11  Inversions          932 non-null    float64
 12  Lift/launch system  795 non-null    object 
 13  Cost                382 non-null    object 
 14  Trains              718 non-null    object 
 15  Park section        487 non-null    object 
 16  Durati

### Fixing Column Names - convert to lowercase, replace separators by _

In [285]:
df.rename(columns={ col: col.lower() for col in df.columns}, inplace=True)
df.rename(columns={ col: col.replace("-", "_") for col in df.columns}, inplace=True)
df.rename(columns={ col: col.replace("/", "_") for col in df.columns}, inplace=True)
df.rename(columns={ col: col.replace(" ", "_") for col in df.columns}, inplace=True)

###  Droping the following 10 columns as those are not required (speed1, speed2, speed1_value, speed1_unit, speed_mph, height_value, height_unit)

In [286]:
df.drop(columns=["opening_date_clean", "speed1", "speed2", "speed1_value", "speed1_unit", "speed_mph", "height_value", "height_unit", "inversions_clean","gforce_clean"], inplace=True)

### Number of columns reduced from 56 to 28 after removing columns with 80% null values and not required columns

In [287]:
df.shape

(1087, 28)

### Total number of fields that don't have value is 9192

In [288]:
df.isna().sum().sum()

9192

## Get the coaster details from Wikipedia

In [289]:
def get_coaster_details(coaster_name):
    headers = {"User-Agent": "Whoama"}
    url = f"https://en.wikipedia.org/wiki/{coaster_name}"
    r = requests.get(url, headers=headers)
    sp = bs(r.text, "html.parser")
    table = sp.select_one("table.infobox")
    data = {}
    data["coaster_name"] = coaster_name
    
    infobox_header = sp.find('th', class_='infobox-header')
    if infobox_header:
        anchor_tag = infobox_header.find('a')
        if anchor_tag:
            anchor_text = infobox_header.find('a').get_text()
            data["location"] = anchor_text
    
    if table is None:
        return data
    
    for row in table.find_all("tr"):
        header = row.find("th")
        value = row.find("td")
        if header and value:
            key = header.get_text(" ", strip=True).lower().replace(" ", "_")
            value_text = (
                value.get_text(" ", strip=True)
                .replace("\xa0", " ")
                .replace("\ufeff", "")
            )

            if " / " in value_text:
                value_text = value_text.split(" / ")[-1]

            # value_text = re.sub(r"\s*\(.*?\)\s*", "", value_text)
            res = re.search(r"\((.*?)\)", value_text)
            if res == None:
                data[key] = value_text
            else:
                data[key] = res[0].replace("(", "").replace(")", "").strip()
    return data


Below code is used to download the roller coaster data from Wiki to enrich our dataset

Code is commented out as we have saved the data from Wiki in a file wiki_data.csv

In [290]:
# wiki_data = {}
# for index, row in df.iterrows():
#     coaster_name = row['coaster_name']
#     data = get_coaster_details(coaster_name)
#     wiki_data[coaster_name] = data
# 
# df_wiki = pd.DataFrame(wiki_data.values())
# 
# df_wiki.to_csv("wiki_data.csv")

### Load the data from Wiki into df_wiki

In [291]:
df_wiki = pd.read_csv("wiki_data.csv", index_col=0)

### Understanding the data loaded from Wiki

In [292]:
df_wiki.head()

,coaster_name,location,park_section,coordinates,status,opening_date,type,manufacturer,designer,model,track_layout,lift/launch_system,height,drop,length,speed,duration,max_vertical_angle,capacity,g-force,closing_date,inversions,trains,opened,closed,general_manager,replaced,replaced_by,Unnamed: 29,cost,height_restriction,designated,website,area,built,architect,part_of,designated nhldcp,reference no.,architectural style,nrhp reference_no.,chisl_no.,sdhl_no.,added_to_nrhp,designated nhl,designated sdhl,builders,date,deaths,non-fatal_injuries,...,operated_by,age_restriction,handicapped_persons,hangul,hanja,revised_romanization,mccune–reischauer,lift_system,public_transit,subcontractor,previous_theme,current_theme,renamed_to,restraint,maximum_height,vehicle,tunnel_length,animatronics,time_zone,fia_grade,operator,major_events,turns,race_lap_record,queue_host,ride_host,trains_built_by,hosts,former_names,host,original_manufacturer,redesigned_by,cars,audio-animatronics,previous_name,sponcor,launches,site_area,tagline,pre-ride_host,official_website,virtual_queue,physical_length,total_length_traversed,1st_launch,2nd_launch,pre-show_hosts,lift_system_#1,lift_system_#2,ride_hosts
0,Switchback Railway,Coney Island,Coney Island Cyclone Site,40.574; -73.978,Removed,1884-06-16,Wood,LaMarcus Adna Thompson,LaMarcus Adna Thompson,Lift Packed,Gravity pulled coaster,gravity,15 m,13 m,180 m,9.7 km/h,1:00,30°,1600 riders per hour,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Flip Flap Railway,Sea Lion Park,NaN,40.578; -73.979,Removed,1895,Wood,Lina Beecher,Lina Beecher,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,1902,1,a single car. Riders are arranged 1 across in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Switchback Railway (Euclid Beach Park),"Cleveland , Ohio , United States",NaN,41.580; -81.570,Defunct,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1895,1969-09-28,Jonson McBronson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Loop the Loop (Coney Island),Coney Island,NaN,40.5745; -73.978,Removed,1901,Steel,Edwin Prescott,Edward A. Green,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1910,1,a single car. Riders are arranged 2 across in ...,NaN,NaN,NaN,Switchback Railway,Giant Racer,Track 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Loop the Loop (Young's Pier),Atlantic City,NaN,39.3538; -74.4342,Removed,1901,Steel,Edwin Prescott,Edward A. Green,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1912,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Returns the coaster details from wiki based on the coaster name

In [293]:
def get_coaster_from_wiki(coaster_name):
    coaster = df_wiki.query("coaster_name == @coaster_name")
    return coaster


 Check if the above function is working fine

In [294]:
get_coaster_from_wiki("Switchback Railway")

,coaster_name,location,park_section,coordinates,status,opening_date,type,manufacturer,designer,model,track_layout,lift/launch_system,height,drop,length,speed,duration,max_vertical_angle,capacity,g-force,closing_date,inversions,trains,opened,closed,general_manager,replaced,replaced_by,Unnamed: 29,cost,height_restriction,designated,website,area,built,architect,part_of,designated nhldcp,reference no.,architectural style,nrhp reference_no.,chisl_no.,sdhl_no.,added_to_nrhp,designated nhl,designated sdhl,builders,date,deaths,non-fatal_injuries,...,operated_by,age_restriction,handicapped_persons,hangul,hanja,revised_romanization,mccune–reischauer,lift_system,public_transit,subcontractor,previous_theme,current_theme,renamed_to,restraint,maximum_height,vehicle,tunnel_length,animatronics,time_zone,fia_grade,operator,major_events,turns,race_lap_record,queue_host,ride_host,trains_built_by,hosts,former_names,host,original_manufacturer,redesigned_by,cars,audio-animatronics,previous_name,sponcor,launches,site_area,tagline,pre-ride_host,official_website,virtual_queue,physical_length,total_length_traversed,1st_launch,2nd_launch,pre-show_hosts,lift_system_#1,lift_system_#2,ride_hosts
0,Switchback Railway,Coney Island,Coney Island Cyclone Site,40.574; -73.978,Removed,1884-06-16,Wood,LaMarcus Adna Thompson,LaMarcus Adna Thompson,Lift Packed,Gravity pulled coaster,gravity,15 m,13 m,180 m,9.7 km/h,1:00,30°,1600 riders per hour,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Returns the value of a particular field from wiki by coaster_name

In [295]:
def get_field_from_wiki_by_coaster_name(coaster_name, field):
    coaster = df_wiki.query("coaster_name == @coaster_name")
    if field in df_wiki.columns:
        return coaster.get(field)
    return None

Check if the above method is working fine

In [296]:
print(get_field_from_wiki_by_coaster_name("Switchback Railway", "status"))

0    Removed
Name: status, dtype: object


In [297]:
df.status.value_counts()

Operating                                                         668
Removed                                                           137
Closed                                                             24
Under construction                                                 15
In Production                                                      11
Discontinued                                                        7
Closed in 2021                                                      2
SBNO December 2019                                                  2
Chapter 7 bankruptcy; rides dismantled and sold; property sold      2
Not Currently Operating                                             1
closed for maintenance as of july 30 no reopening date known        1
Temporarily Closed                                                  1
SBNO (Standing But Not Operating)                                   1
Temporarily closed                                                  1
Under Maintenance   

### Retain only operating roller coasters for our analysis - 668 records

In [298]:
df = df.query("status == 'Operating'")
df.shape

(668, 28)

In [299]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668 entries, 6 to 1071
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   coaster_name        668 non-null    object 
 1   length              616 non-null    object 
 2   speed               608 non-null    object 
 3   location            668 non-null    object 
 4   status              668 non-null    object 
 5   opening_date        657 non-null    object 
 6   type                668 non-null    object 
 7   manufacturer        644 non-null    object 
 8   height_restriction  571 non-null    object 
 9   model               463 non-null    object 
 10  height              616 non-null    object 
 11  inversions          603 non-null    float64
 12  lift_launch_system  520 non-null    object 
 13  cost                280 non-null    object 
 14  trains              484 non-null    object 
 15  park_section        365 non-null    object 
 16  duratio

Since we have only Operating roller coaster, the closing date should be na

However we could see that there are around 9 rows that have a value in closing date

In [300]:
(~df['closing_date'].isna()).sum()

9

Checked the cases where the closing_date was present and it was because the roller coaster was closed and then reopened

Hence dropping the column as it is of no use to us

In [301]:
df.drop(columns='closing_date', axis=1, inplace=True)

In [302]:
df.shape

(668, 27)

In [303]:
df.head()

,coaster_name,length,speed,location,status,opening_date,type,manufacturer,height_restriction,model,height,inversions,lift_launch_system,cost,trains,park_section,duration,capacity,g_force,designer,max_vertical_angle,drop,track_layout,year_introduced,latitude,longitude,type_main
6,Leap-The-Dips,"1,452 ft (443 m)",10 mph (16 km/h),Lakemont Park,Operating,1902,Wood – Side friction,Federal Construction Company,NaN,NaN,41 ft (12 m),NaN,NaN,NaN,NaN,NaN,1:00,NaN,NaN,Edward Joy Morris,25°,9 ft (2.7 m),NaN,1902,NaN,NaN,Wood
13,Racer (1910 roller coaster),"4,500 ft (1,400 m)",40 mph (64 km/h),Kennywood,Operating,1927,Wood – Racing,Charlie Mach,46 in (117 cm),Racing,72.5 ft (22.1 m),0.0,Chain lift,"$75,000",4 trains with 4 cars. Riders are arranged 2 ac...,NaN,1:32,1400 riders per hour,NaN,John A. Miller,NaN,50 ft (15 m),Möbius Loop,1910,40.3869,-79.8619,Wood
15,The Great Scenic Railway,"967 m (3,173 ft)",60 km/h (37 mph),Luna Park Melbourne,Operating,December 1912,Wood,LaMarcus Thompson,100 cm (3 ft 3 in),NaN,16 m (52 ft),0.0,Cable lift hill,NaN,3 trains with 2 cars. Riders are arranged 2 ac...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1912,NaN,NaN,Wood
22,Jack Rabbit (Kennywood),"2,132 ft (650 m)",45 mph (72 km/h),Kennywood,Operating,1920,Wood,Harry C. Baker,42 in (107 cm),NaN,40 ft (12 m),0.0,Chain lift hill,"$50,000 USD",3 trains with 3 cars. Riders are arranged 2 ac...,NaN,1:36,NaN,NaN,John A. Miller,45°,70 ft (21 m),"Terrain, Out and Back roller coaster",1920,40.3867,-79.8628,Wood
23,Jack Rabbit (Seabreeze),"2,130 ft (650 m)",42 mph (68 km/h),Seabreeze Amusement Park,Operating,1920,Wood,Harry C. Baker,48 in (122 cm),NaN,75 ft (23 m),NaN,chain,NaN,NaN,NaN,NaN,NaN,NaN,John A. Miller,52°,NaN,"Terrain, Out and Back",1920,43.2316,-77.5432,Wood


 Check for incorrect rows / summary rows etc
 No such rows were found

In [304]:
# Check for duplicated rows
df.loc[df.duplicated()]

,coaster_name,length,speed,location,status,opening_date,type,manufacturer,height_restriction,model,height,inversions,lift_launch_system,cost,trains,park_section,duration,capacity,g_force,designer,max_vertical_angle,drop,track_layout,year_introduced,latitude,longitude,type_main


In [305]:
# Check if there are duplicate records for coaster_name
coaster = df['coaster_name'].value_counts().reset_index(name="count")

In [306]:
coaster[coaster['count']>1]

,index,count
0,Journey to Atlantis,3
1,Lil' Devil Coaster,2
2,The Barnstormer,2
3,Untamed (Walibi Holland),2
4,Twisted Cyclone,2
5,Wipeout (roller coaster),2
6,Kraken (roller coaster),2
7,Hades 360,2
8,Incredicoaster,2
9,Scorpion Express,2


 - There are 3 occurences of the coaster name - Journey to Atlantis	
 
 - Looking at the data we could see that the year_introduced is the only one that changes
 
 - We can keep just the first row as that would be having the correct value of year_introduced 

 - Removing the other rows as they are duplicates

In [307]:
df[df['coaster_name'] == "Journey to Atlantis"]

,coaster_name,length,speed,location,status,opening_date,type,manufacturer,height_restriction,model,height,inversions,lift_launch_system,cost,trains,park_section,duration,capacity,g_force,designer,max_vertical_angle,drop,track_layout,year_introduced,latitude,longitude,type_main
444,Journey to Atlantis,NaN,NaN,SeaWorld Orlando,Operating,"April 17, 1998",Steel,Mack Rides,42 in (107 cm),Water Coaster,NaN,0.0,Chain lift hill,NaN,NaN,Sea of Legends,5:56,NaN,NaN,NaN,NaN,60 ft (18 m),Water Coaster,1998,NaN,NaN,Steel
653,Journey to Atlantis,NaN,NaN,SeaWorld Orlando,Operating,"April 17, 1998",Steel,Mack Rides,42 in (107 cm),Water Coaster,NaN,0.0,Chain lift hill,NaN,NaN,Sea of Legends,5:56,NaN,NaN,NaN,NaN,60 ft (18 m),Water Coaster,2004,NaN,NaN,Steel
732,Journey to Atlantis,NaN,NaN,SeaWorld Orlando,Operating,"April 17, 1998",Steel,Mack Rides,42 in (107 cm),Water Coaster,NaN,0.0,Chain lift hill,NaN,NaN,Sea of Legends,5:56,NaN,NaN,NaN,NaN,60 ft (18 m),Water Coaster,2007,NaN,NaN,Steel


Deleting duplicated rows

In [308]:
df_deduped = df[~df.drop(columns=['year_introduced']).duplicated(keep="first")].sort_values(by="coaster_name")

Total of 22 records that were duplicate have been deleted

In [309]:
print(df.shape)
print(df_deduped.shape)

(668, 27)
(646, 27)


Cross check and ensure there are no duplicate coaster_name

In [310]:
coaster = df_deduped['coaster_name'].value_counts().reset_index(name="count")
coaster[coaster['count']>1]

,index,count


In [311]:
df_deduped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 646 entries, 696 to 434
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   coaster_name        646 non-null    object 
 1   length              598 non-null    object 
 2   speed               591 non-null    object 
 3   location            646 non-null    object 
 4   status              646 non-null    object 
 5   opening_date        635 non-null    object 
 6   type                646 non-null    object 
 7   manufacturer        622 non-null    object 
 8   height_restriction  552 non-null    object 
 9   model               444 non-null    object 
 10  height              598 non-null    object 
 11  inversions          582 non-null    float64
 12  lift_launch_system  501 non-null    object 
 13  cost                271 non-null    object 
 14  trains              467 non-null    object 
 15  park_section        352 non-null    object 
 16  durati

In [312]:
df_deduped.head()

,coaster_name,length,speed,location,status,opening_date,type,manufacturer,height_restriction,model,height,inversions,lift_launch_system,cost,trains,park_section,duration,capacity,g_force,designer,max_vertical_angle,drop,track_layout,year_introduced,latitude,longitude,type_main
696,10 Inversion Roller Coaster,"2,788.8 ft (850.0 m)",45 mph (72 km/h),Chimelong Paradise,Operating,2006,Steel,Intamin,54–77 in (137–196 cm),Looping Coaster (Colossus),98.4 ft (30.0 m),10.0,NaN,NaN,Single train with 7 cars. Riders are arranged ...,NaN,1:32,"1,300 riders per hour",NaN,Werner Stengel,NaN,NaN,NaN,2006,23.0044,113.3269,Steel
886,Abyss (roller coaster),"630 m (2,070 ft)",85 km/h (53 mph),Adventure World,Operating,1 November 2013,Steel – Euro-Fighter,Gerstlauer,125 cm (4 ft 1 in),Euro-Fighter – Custom,30 m (98 ft),3.0,Chain lift hill,A$12 million,4 trains with a single car. Riders are arrange...,NaN,2 minutes,NaN,4.5,NaN,100°,30.5 m (100 ft),Custom,2013,-32.0955,115.8170,Steel
1057,Abyssus,"4,317.6 ft (1,316.0 m)",62.1 mph (99.9 km/h),Energylandia,Operating,"July 14, 2021",Steel – Launched,Vekoma,120 cm (3 ft 11 in),Shockwave 1320m+,126.3 ft (38.5 m),4.0,LSM,"€11,000,000",3 trains with 4 cars. Riders are arranged 2 ac...,Aqualantis,NaN,1000 riders per hour,4.5,Benjamin Bloemendaal,NaN,NaN,NaN,2021,49.9992,19.4011,Steel
828,Accelerator (roller coaster),180 m (590 ft),60 km/h (37 mph),Drayton Manor,Operating,21 April 2011,Steel – Junior – Boomerang,Vekoma,100 cm (3 ft 3 in),Family Boomerang,20 m (66 ft),0.0,Booster Wheel Lift Hill,"£2,100,000",NaN,Aerial Park,NaN,750 riders per hour,3,Meticulous ltd,NaN,NaN,NaN,2011,52.6108,-1.7134,Steel
936,Acrobat (roller coaster),"3,349.8 ft (1,021.0 m)",55.9 mph (90.0 km/h),Nagashima Spa Land,Operating,"July 18, 2015",Steel – Flying,Bolliger & Mabillard,135–198 cm (4 ft 5 in–6 ft 6 in),Flying Coaster (Manta),141.1 ft (43.0 m),4.0,Chain lift hill,NaN,NaN,NaN,3:00,NaN,NaN,NaN,NaN,NaN,NaN,2015,NaN,NaN,Steel


- Remove ft from the column

- Remove the meter part as well

- Change the column type to float64

In [413]:
df_deduped['length(m)'] = df_deduped['length'].str.extract(r'([\d,\.]+)\s*m').replace(',', '', regex=True).astype(float)

In [314]:
df_deduped[["length", "length(m)"]]

,length,length(m)
696,"2,788.8 ft (850.0 m)",850.0
886,"630 m (2,070 ft)",630.0
1057,"4,317.6 ft (1,316.0 m)",1316.0
828,180 m (590 ft),180.0
936,"3,349.8 ft (1,021.0 m)",1021.0
...,...,...
364,NaN,NaN
1046,"1,316 m (4,318 ft)",1316.0
433,"2,900 ft (880 m)",880.0
56,NaN,NaN


In [315]:
df_deduped['length(m)'].unique()

array([ 850.  ,  630.  , 1316.  ,  180.  , 1021.  ,  320.  ,  903.  ,
        901.  ,     nan,  264.  , 1167.  ,  719.  , 1417.3 ,  827.  ,
       1200.  ,  820.  ,  877.  , 1488.  ,  400.  , 1020.  ,  550.  ,
        670.  ,  354.  ,  207.  , 1070.  , 1099.  , 1560.  , 1257.  ,
        501.  ,  342.  ,  870.  ,  689.  , 1620.9 , 1456.  , 1000.  ,
       1215.  ,  768.  , 1056.  ,  780.  ,  765.  ,  986.  ,  285.  ,
        594.  , 1440.  , 1851.  , 1413.  ,  834.  ,  693.  ,  530.  ,
        739.  ,  640.  ,  731.  ,  360.  ,  770.  , 1350.  ,  408.  ,
       1280.  , 1344.  , 1520.  ,  662.  ,  800.  ,  992.  ,  620.  ,
        590.  ,  135.  , 1174.  ,  390.  ,  853.  , 1010.  ,  580.  ,
       1400.  ,  709.  , 1193.  ,  490.  ,  420.  ,  520.  ,  525.  ,
       1610.  ,  794.  , 1050.  ,  972.  ,  380.  ,  270.  , 1036.  ,
         80.  ,  658.5 , 1285.  , 1700.  ,  230.  ,  453.  ,  361.  ,
        600.  ,  564.  , 1153.01,  725.1 ,  263.  ,  350.  ,  700.  ,
        980.  ,  395

In [316]:
df_deduped['length(m)'] = df_deduped['length(m)'].astype(float)

- Remove mph from the column

- Remove the km/h part as well

- Change the column type to float64

In [317]:
df_deduped["speed(km/h)"] = df_deduped['speed'].str.extract(r'([\d,\.]+)\s*km/h').replace(',', '', regex=True).astype(float)

In [318]:
df_deduped[["speed", "speed(km/h)"]].head(5)

,speed,speed(km/h)
696,45 mph (72 km/h),72.0
886,85 km/h (53 mph),85.0
1057,62.1 mph (99.9 km/h),99.9
828,60 km/h (37 mph),60.0
936,55.9 mph (90.0 km/h),90.0


In [319]:
df_deduped['speed(km/h)'].unique()

array([ 72.  ,  85.  ,  99.9 ,  60.  ,  90.  ,  56.  , 100.  ,    nan,
        45.  , 108.  ,  83.  , 106.2 ,  77.  ,  80.  ,  80.6 , 117.  ,
        31.  ,  75.  ,  88.  ,  34.9 , 109.9 , 109.  ,  89.  ,  82.1 ,
       124.  ,  64.  ,  98.  ,  82.  ,  50.  ,  76.  ,  97.  , 122.  ,
       110.  ,  55.  ,  66.  ,  16.  ,  48.  ,  36.  ,  68.  ,  70.  ,
        50.1 ,  92.  ,  40.  ,  46.5 ,  60.7 ,  93.  , 115.1 ,  47.  ,
        84.  ,  40.1 ,  69.  ,  68.4 , 130.  ,  78.1 , 126.  , 110.1 ,
       104.6 ,  73.4 ,  45.9 ,  41.8 ,  87.7 , 120.1 ,  46.8 ,  58.  ,
        48.9 ,  95.  ,  43.  ,  55.5 ,  79.99,  42.  , 120.  ,  27.  ,
        39.  , 240.  ,  79.2 , 135.  , 106.6 , 153.  ,  96.6 ,  87.  ,
        86.4 , 116.  , 137.  , 113.  , 106.  ,  18.  , 114.  ,  42.6 ,
       107.  , 105.  ,  26.1 ,  30.  , 142.  ,  85.9 , 121.  , 145.  ,
       101.  ,  48.4 , 104.4 ,  91.  , 206.  , 103.  ,  79.  ,  88.4 ,
        96.1 , 148.  ,  32.  ,  53.  ,  43.5 ,  89.9 , 150.  ,  61.  ,
      

In [320]:
df_deduped.location.value_counts()

Other                         20
Six Flags Magic Mountain      15
Cedar Point                   14
Kings Island                  14
Carowinds                     13
                              ..
Ocean Park Hong Kong           1
Attractiepark Slagharen        1
Santa Cruz Beach Boardwalk     1
Galaxyland                     1
ZDT's Amusement Park           1
Name: location, Length: 222, dtype: int64

There are 20 rows where the location is Other

In [321]:
len(df_deduped.query("location == 'Other'"))

20

In [322]:
def get_location_by_coaster_name(row):
    if row['location'] == None or row['location'] == "Other":
        row['location'] = get_field_from_wiki_by_coaster_name("location", row['coaster_name'])
    return row

### Update the location that is having the value of Other by fetching it using the coaster name from Wiki 

In [323]:
df_deduped.loc[df_deduped['location'] == 'Other', 'location'] = df_deduped.query("location == 'Other'").apply(lambda x: get_location_by_coaster_name(x), axis=1)

As expected, there are no locations with Other after replacement

In [324]:
df_deduped.query("location == 'Other'")

,coaster_name,length,speed,location,status,opening_date,type,manufacturer,height_restriction,model,height,inversions,lift_launch_system,cost,trains,park_section,duration,capacity,g_force,designer,max_vertical_angle,drop,track_layout,year_introduced,latitude,longitude,type_main,length(m),speed(km/h)


 There is still one coaster for which the location is None

In [325]:
df_deduped.location.isna().sum()

20

In [326]:
df_deduped.location.unique()

array(['Chimelong Paradise', 'Adventure World', 'Energylandia',
       'Drayton Manor', 'Nagashima Spa Land', 'Oaks Amusement Park',
       'Kings Island', 'Carowinds', 'Tampa, Florida, U.S.', 'Europa-Park',
       'Busch Gardens Williamsburg', 'Nigloland', None,
       'Six Flags St. Louis', 'Walygator Parc', 'Kings Dominion',
       'Six Flags Magic Mountain', 'Magic Springs and Crystal Falls',
       'Lotte World', 'Xetulul Theme Park', 'Blackpool Pleasure Beach',
       'Nickelodeon Universe', 'Movie Park Germany', 'Liseberg',
       'Yomiuriland', 'Efteling', 'Lagoon Amusement Park',
       'Parque Warner Madrid', 'Six Flags New England',
       'Six Flags Mexico', 'Six Flags America', "Canada's Wonderland",
       'New York-New York Hotel and Casino', 'Camden Park',
       'Six Flags Great Adventure', 'Knoebels Amusement Resort',
       'Phantasialand', 'Cedar Point', 'Gardaland', 'Kemah Boardwalk',
       'Lagoon', 'Worlds of Fun', 'Toverland', 'Lake Compounce',
       'Disney C

In [327]:
df_deduped.head()

,coaster_name,length,speed,location,status,opening_date,type,manufacturer,height_restriction,model,height,inversions,lift_launch_system,cost,trains,park_section,duration,capacity,g_force,designer,max_vertical_angle,drop,track_layout,year_introduced,latitude,longitude,type_main,length(m),speed(km/h)
696,10 Inversion Roller Coaster,"2,788.8 ft (850.0 m)",45 mph (72 km/h),Chimelong Paradise,Operating,2006,Steel,Intamin,54–77 in (137–196 cm),Looping Coaster (Colossus),98.4 ft (30.0 m),10.0,NaN,NaN,Single train with 7 cars. Riders are arranged ...,NaN,1:32,"1,300 riders per hour",NaN,Werner Stengel,NaN,NaN,NaN,2006,23.0044,113.3269,Steel,850.0,72.0
886,Abyss (roller coaster),"630 m (2,070 ft)",85 km/h (53 mph),Adventure World,Operating,1 November 2013,Steel – Euro-Fighter,Gerstlauer,125 cm (4 ft 1 in),Euro-Fighter – Custom,30 m (98 ft),3.0,Chain lift hill,A$12 million,4 trains with a single car. Riders are arrange...,NaN,2 minutes,NaN,4.5,NaN,100°,30.5 m (100 ft),Custom,2013,-32.0955,115.8170,Steel,630.0,85.0
1057,Abyssus,"4,317.6 ft (1,316.0 m)",62.1 mph (99.9 km/h),Energylandia,Operating,"July 14, 2021",Steel – Launched,Vekoma,120 cm (3 ft 11 in),Shockwave 1320m+,126.3 ft (38.5 m),4.0,LSM,"€11,000,000",3 trains with 4 cars. Riders are arranged 2 ac...,Aqualantis,NaN,1000 riders per hour,4.5,Benjamin Bloemendaal,NaN,NaN,NaN,2021,49.9992,19.4011,Steel,1316.0,99.9
828,Accelerator (roller coaster),180 m (590 ft),60 km/h (37 mph),Drayton Manor,Operating,21 April 2011,Steel – Junior – Boomerang,Vekoma,100 cm (3 ft 3 in),Family Boomerang,20 m (66 ft),0.0,Booster Wheel Lift Hill,"£2,100,000",NaN,Aerial Park,NaN,750 riders per hour,3,Meticulous ltd,NaN,NaN,NaN,2011,52.6108,-1.7134,Steel,180.0,60.0
936,Acrobat (roller coaster),"3,349.8 ft (1,021.0 m)",55.9 mph (90.0 km/h),Nagashima Spa Land,Operating,"July 18, 2015",Steel – Flying,Bolliger & Mabillard,135–198 cm (4 ft 5 in–6 ft 6 in),Flying Coaster (Manta),141.1 ft (43.0 m),4.0,Chain lift hill,NaN,NaN,NaN,3:00,NaN,NaN,NaN,NaN,NaN,NaN,2015,NaN,NaN,Steel,1021.0,90.0


----------------------

- Some opening_date are having None values or just have the year values 

- For such rows we get the date from the wiki 

- and we replace the \xa0 value by space to clean the values

In [328]:
def get_opening_date_by_coaster_name(row):
    opening_date = str(row["opening_date"])
    if opening_date == None or len(opening_date) == 4:
        date = get_field_from_wiki_by_coaster_name("opening_date", row['coaster_name'])
        row['opening_date'] = date
    else:
        date = opening_date.replace("\xa0", " ")
        row['opening_date'] = date
    return row

In [329]:
df_deduped.loc[:, 'opening_date'] = df_deduped.apply(lambda x: get_opening_date_by_coaster_name(x), axis=1)

### The date fields have lot of extra data like [1] and [ 1 ] etc which we need to remove and also finally convert them into DateTime format

In [330]:
def convert_date(date):
    if date == None:
        return None
    orig_date = date
    try:
        date = date.replace("[1]", "").replace("[2]", "").replace("[ 1 ]", "").replace("[ 2 ]", "").replace(",", "").strip()
        date = date.split(";")[0]
        parts = date.split(" ")
        if len(parts) > 3:
            date = parts[0] + " " + parts[1] + " " +parts[2][:4]
        if date == None:
            return None
        res = pd.to_datetime(date)
        return res
    except:
        match = re.search(r"\d{4}", orig_date)
        if match:
            date = match.group()
            res = pd.to_datetime(date)
        elif orig_date and len(orig_date) > 4:
            date = orig_date[:4]
            print(orig_date, date)
            res = pd.to_datetime(date)
        else:
            res = None
        return res


In [331]:
df_deduped['opening_date'] = df_deduped['opening_date'].map(lambda x: convert_date(str(x)))

In [332]:
list(df_deduped['opening_date'].unique())

[numpy.datetime64('NaT'),
 numpy.datetime64('2013-11-01T00:00:00.000000000'),
 numpy.datetime64('2021-07-14T00:00:00.000000000'),
 numpy.datetime64('2011-04-21T00:00:00.000000000'),
 numpy.datetime64('2015-07-18T00:00:00.000000000'),
 numpy.datetime64('2018-03-24T00:00:00.000000000'),
 numpy.datetime64('1991-04-13T00:00:00.000000000'),
 numpy.datetime64('1999-03-20T00:00:00.000000000'),
 numpy.datetime64('1997-03-22T00:00:00.000000000'),
 numpy.datetime64('2014-04-12T00:00:00.000000000'),
 numpy.datetime64('1981-05-23T00:00:00.000000000'),
 numpy.datetime64('2008-06-20T00:00:00.000000000'),
 numpy.datetime64('1991-03-23T00:00:00.000000000'),
 numpy.datetime64('2009-05-23T00:00:00.000000000'),
 numpy.datetime64('1999-03-30T00:00:00.000000000'),
 numpy.datetime64('2002-03-22T00:00:00.000000000'),
 numpy.datetime64('2014-06-01T00:00:00.000000000'),
 numpy.datetime64('1988-06-22T00:00:00.000000000'),
 numpy.datetime64('1996-06-30T00:00:00.000000000'),
 numpy.datetime64('2003-04-12T00:00:00

### Again the type column has a lot of values wit this \xa0 which we need to clean 

In [333]:
df_deduped.type.unique()

array(['Steel', 'Steel\xa0– Euro-Fighter', 'Steel\xa0– Launched',
       'Steel\xa0– Junior\xa0– Boomerang', 'Steel\xa0– Flying',
       'Steel\xa0– Mine Train', 'Steel\xa0– Inverted', 'Other',
       'Steel\xa0– Powered\xa0– Virtual reality', 'Wood\xa0– Racing',
       'Wood', 'Steel\xa0– Wild Mouse', 'Wood\xa0– Out and back',
       'Steel\xa0– Inverted\xa0– Powered', 'Steel\xa0– Bobsled',
       'Steel\xa0– Powered\xa0– Indoor', 'Steel\xa0– Junior',
       'Steel\xa0– Dive Coaster', 'Steel\xa0– Floorless Coaster',
       'Steel\xa0– Dueling\xa0– Launched',
       'Steel\xa0– Family\xa0– Enclosed', 'Wood\xa0– Twister',
       'Steel\xa0– Boomerang\xa0– Shuttle',
       'Steel\xa0– Motorbike\xa0– Launched', 'Steel\xa0– Suspended',
       'Steel\xa0– Indoor', 'Steel\xa0– Family', 'Steel\xa0– Shuttle',
       'Steel\xa0– Spinning', 'Steel\xa0– Enclosed',
       'Steel\xa0– Spinning\xa0– Enclosed',
       'Steel\xa0– Dueling\xa0– Racing', 'Steel\xa0– 4th Dimension',
       'Steel\xa0– 4t

In [334]:
df_deduped.loc[:, 'type'] = df_deduped.type.map(lambda x : x.split(" ")[0]).replace("–", "", regex=True).str.strip()

### Now we have the cleeaned type values

In [335]:
df_deduped.type.value_counts()

Steel    492
Wood     131
Other     23
Name: type, dtype: int64

In [336]:
df_deduped.type.unique()

array(['Steel', 'Other', 'Wood'], dtype=object)

In [337]:
df_deduped.manufacturer.isna().sum()

24

In [338]:
df_deduped.head()

,coaster_name,length,speed,location,status,opening_date,type,manufacturer,height_restriction,model,height,inversions,lift_launch_system,cost,trains,park_section,duration,capacity,g_force,designer,max_vertical_angle,drop,track_layout,year_introduced,latitude,longitude,type_main,length(m),speed(km/h)
696,10 Inversion Roller Coaster,"2,788.8 ft (850.0 m)",45 mph (72 km/h),Chimelong Paradise,Operating,NaT,Steel,Intamin,54–77 in (137–196 cm),Looping Coaster (Colossus),98.4 ft (30.0 m),10.0,NaN,NaN,Single train with 7 cars. Riders are arranged ...,NaN,1:32,"1,300 riders per hour",NaN,Werner Stengel,NaN,NaN,NaN,2006,23.0044,113.3269,Steel,850.0,72.0
886,Abyss (roller coaster),"630 m (2,070 ft)",85 km/h (53 mph),Adventure World,Operating,2013-11-01,Steel,Gerstlauer,125 cm (4 ft 1 in),Euro-Fighter – Custom,30 m (98 ft),3.0,Chain lift hill,A$12 million,4 trains with a single car. Riders are arrange...,NaN,2 minutes,NaN,4.5,NaN,100°,30.5 m (100 ft),Custom,2013,-32.0955,115.8170,Steel,630.0,85.0
1057,Abyssus,"4,317.6 ft (1,316.0 m)",62.1 mph (99.9 km/h),Energylandia,Operating,2021-07-14,Steel,Vekoma,120 cm (3 ft 11 in),Shockwave 1320m+,126.3 ft (38.5 m),4.0,LSM,"€11,000,000",3 trains with 4 cars. Riders are arranged 2 ac...,Aqualantis,NaN,1000 riders per hour,4.5,Benjamin Bloemendaal,NaN,NaN,NaN,2021,49.9992,19.4011,Steel,1316.0,99.9
828,Accelerator (roller coaster),180 m (590 ft),60 km/h (37 mph),Drayton Manor,Operating,2011-04-21,Steel,Vekoma,100 cm (3 ft 3 in),Family Boomerang,20 m (66 ft),0.0,Booster Wheel Lift Hill,"£2,100,000",NaN,Aerial Park,NaN,750 riders per hour,3,Meticulous ltd,NaN,NaN,NaN,2011,52.6108,-1.7134,Steel,180.0,60.0
936,Acrobat (roller coaster),"3,349.8 ft (1,021.0 m)",55.9 mph (90.0 km/h),Nagashima Spa Land,Operating,2015-07-18,Steel,Bolliger & Mabillard,135–198 cm (4 ft 5 in–6 ft 6 in),Flying Coaster (Manta),141.1 ft (43.0 m),4.0,Chain lift hill,NaN,NaN,NaN,3:00,NaN,NaN,NaN,NaN,NaN,NaN,2015,NaN,NaN,Steel,1021.0,90.0


## We need to standardise the height_restriction values into centimeters to keep it easy for further use

In [339]:
def get_height_restriction(height_restriction):
    height_restriction = height_restriction.replace("\xa0", "").replace("[2]", "").replace("[3]", "").replace("[4]", "")
    res = re.search(r"(\d+–\d+\s*cm)", height_restriction)
    if res != None:
        return res[0].replace("(", "").replace(")", "").replace("–", ",").replace("cm", "").strip().split(",")[0]
    elif re.search(r"(\d+)\s*ft\s*(\d+)\s*in", height_restriction):
        match = re.search(r"(\d+)\s*ft\s*(\d+)\s*in", height_restriction)
        feet = match.group(1)
        inch = match.group(2)
        res = round(float(feet) * 30.48 + float(inch) * 2.54, 2)
        print(feet, inch, res)
        return res
    else:
        res = re.search(r"\d+\s*cm", height_restriction)
        if res != None:
            return res[0].replace("cm", "").strip()
        else:
            return height_restriction

In [340]:
df_deduped.loc[:, "height_restriction_clean"] = df_deduped.height_restriction.map(lambda x : get_height_restriction(str(x)))

4 1 124.46
3 11 119.38
3 3 99.06
3 3 99.06
3 3 99.06
3 3 99.06
4 3 129.54
3 11 119.38
3 6 106.68
4 3 129.54
3 3 99.06
3 11 119.38
4 7 139.7
4 0 121.92
3 11 119.38
3 3 99.06
3 11 119.38
4 4 132.08
3 0 91.44
3 7 109.22
3 11 119.38
4 3 129.54
3 11 119.38
4 3 129.54
2 11 88.9
4 3 129.54
4 7 139.7
3 4 101.6
4 3 129.54
4 7 139.7
4 7 139.7
4 7 139.7
4 7 139.7
4 3 129.54
4 4 132.08
4 3 129.54
4 7 139.7
4 1 124.46
3 7 109.22
4 3 129.54
3 11 119.38
4 1 124.46
3 11 119.38
4 7 139.7
3 7 109.22
3 7 109.22
4 7 139.7
3 3 99.06
3 11 119.38
3 7 109.22
4 0 121.92
3 11 119.38
4 7 139.7
4 7 139.7
4 7 139.7
4 7 139.7
4 7 139.7
2 11 88.9
4 3 129.54
3 5 104.14
3 3 99.06
4 7 139.7
4 3 129.54
3 3 99.06
4 3 129.54
3 11 119.38
3 3 99.06
4 7 139.7
3 7 109.22
2 11 88.9
2 11 88.9
4 7 139.7
3 7 109.22
2 11 88.9
4 7 139.7
3 4 101.6
4 1 124.46
3 11 119.38
4 3 129.54
3 3 99.06
3 7 109.22
4 3 129.54
4 7 139.7
4 7 139.7
4 1 124.46
4 3 129.54
3 3 99.06
4 3 129.54
4 3 129.54
4 7 139.7
4 7 139.7
4 3 129.54
3 4 101.6
4 3 129

In [341]:
df_deduped[["height_restriction", "height_restriction_clean"]]

,height_restriction,height_restriction_clean
696,54–77 in (137–196 cm),137
886,125 cm (4 ft 1 in),124.46
1057,120 cm (3 ft 11 in),119.38
828,100 cm (3 ft 3 in),99.06
936,135–198 cm (4 ft 5 in–6 ft 6 in),135
...,...,...
364,40 in (102 cm),102
1046,140 cm (4 ft 7 in),139.7
433,48 in (122 cm),122
56,NaN,nan


In [342]:
df_deduped['height_restriction_clean'].unique()

array(['137', 124.46, 119.38, 99.06, '135', '122', 'nan', '132', '112',
       129.54, '120', '107', '130', '125', '117', '140', '91', '92', '84',
       '104', 106.68, 139.7, 121.92, '99', 132.08, 91.44, 109.22, '109',
       '102', 88.9, '110', 101.6, '127', '97', 104.14, '145', '142', '89',
       137.16], dtype=object)

In [343]:
df_deduped['height_restriction_clean'] = df_deduped['height_restriction_clean'].astype(float)

Standardising height column into meters

In [344]:
def get_height(height):
    res = re.search(r"\(\d+.\d+\s*m\)", height)
    if res != None:
        # height contains 72 ft (22 m)
        return res[0].replace("(", "").replace(")", "").replace("m", "").replace(",", "").strip()
    else:
        res = re.search(r"\d+.?\d*\s*m", height)
        if res != None:
            # height contains 30 m (98 ft)
            return res[0].replace("(", "").replace(")", "").replace("m", "").replace(",", "").strip()
        else:
            # height contains only ft - 119.8 ft
            if "ft" in height:
                height = height.replace("ft", "")
                return str(round(float(height) * 0.3048, 2))
            return height

In [345]:
df_deduped['height_clean'] = df_deduped.height.map(lambda x: get_height(str(x)))

In [346]:
df_deduped[['height', 'height_clean']]

,height,height_clean
696,98.4 ft (30.0 m),30.0
886,30 m (98 ft),30
1057,126.3 ft (38.5 m),38.5
828,20 m (66 ft),20
936,141.1 ft (43.0 m),43.0
...,...,...
364,NaN,nan
1046,62.8 m (206 ft),62.8
433,90 ft (27 m),27
56,15 ft (4.6 m),4.6


In [347]:
df_deduped.height_clean.unique()

array(['30.0', '30', '38.5', '20', '43.0', '22', '19', '34', 'nan', '6',
       '59', '33', '38.7', '25', '36.0', '39', '29', '52', '16', '13.5',
       '22.0', '18', '8.5', '36', '27.8', '51.0', '51', '14.8', '32',
       '35.9', '33.3', '42.7', '35', '70', '62', '43', '11', '26.0', '38',
       '4.6', '24', '33.00', '14', '15', '37', '64', '63', '21', '2.4',
       '13', '8.00', '31', '35.5', '26', '50.0', '17', '11.7', '61.6',
       '32.0', '25.0', '17.50', '32.99', '69.0', '60', '65', '4', '23.8',
       '45.1', '57', '15.5', '19.3', '28', '76.00', '27', '24.5', '46',
       '14.9', '40', '23', '25.5', '61', '53', '13.0', '12', '11.8',
       '20.0', '33.31', '6.1', '24.7', '19.6', '79', '49', '99', '45',
       '38.1', '12.5', '33.0', '53.3', '50', '72', '47.17', '3.0', '4.0',
       '7.3', '41', '35.0', '55', '8.0', '44', '3.3', '4.5', '77', '27.0',
       '71', '93', '14.0', '40.0', '42', '31.0', '8', '139', '25.4', '47',
       '28.0', '35.89', '39.9', '93.3', '7', '13.75', '9

In [348]:
df_deduped['height_clean'] = df_deduped.height_clean.astype(float)

Standardising all duration to seconds

In [349]:
def convert_duration_to_seconds(duration):
    origDuration = duration
    result = duration
    try:
        duration = duration.replace("~", "").replace("approx", "").strip().lower()
        duration = duration.replace("[1]", ":").replace("[2]", "").replace("[3]", "").strip()
        duration = duration.replace(".", ":").replace("and", "").strip()
        duration = duration.split("[")[0]
        if re.search(r"(\d+):(\d+)", duration):
            # duration 11:30
            duration = duration.replace("minutes", "").replace("minute", "").replace("mins", "").replace("min", "").strip()
            duration = re.search(r"(\d+):(\d+)\s*", duration)[0]
            parts = duration.split(":")
            result = int(parts[0]) * 60 + int(parts[1])
        elif re.search(r"(\d+)\s*min\s*(\d+)\s*sec", duration):
            # duration 1 min 30 sec
            match = re.search(r"(\d+)\s*min\s*(\d+)\s*sec", duration)
            minutes = int(match.group(1))
            seconds = int(match.group(2))
            result = (minutes * 60) + seconds
        elif re.search(r"(\d+)\s*minutes?\s*(\d+)\s*seconds?", duration):
            match = re.search(r"(\d+)\s*minutes?\s*(\d+)\s*seconds?", duration)
            minutes = int(match.group(1))
            seconds = int(match.group(2))
            result = (minutes * 60) + seconds
        elif "minutes" in duration or "minute" in duration:
            duration = duration.replace("minutes", "").replace("minute", "").replace(":", "").strip()
            result = int(duration) * 60
        elif "seconds" in duration or "secs" in duration or  "sec:" in duration:
            duration = duration.replace("seconds", "").replace("secs", "").replace("sec:", "").strip()
            result = int(duration)
        
#         print(duration, " => " ,result)
    except:
        print("in exception: ", origDuration, duration)
    return result

In [350]:
df_deduped.duration.map(lambda x: convert_duration_to_seconds(str(x))).unique()

in exception:  Under two minutes under two
in exception:  7-8 minutes 7-8


array([92, 120, 'nan', 180, 140, 167, 100, 190, 143, 130, 110, 135, 240,
       82, 90, 44, 128, 138, 160, 75, 96, 122, 148, 60, 105, 102, 108, 68,
       150, 156, 146, 52, 137, 170, 175, 145, 220, 84, 144, 51, 55, 115,
       171, 223, 116, 210, 83, 106, 73, 45, 66, 273, 200, 118, 85, 42,
       195, 139, 88, 112, 27, 63, 216, 77, 80, 67, 65, 300, 151, 99, 155,
       161, 213, 32, 356, 172, 154, 142, 70, 28, 'Under two minutes', 174,
       104, 208, 192, 40, 78, 153, 165, 127, 23, 36, 50, 103, 205, 87,
       134, 117, 95, 101, 113, 98, 136, '66', 25, 58, 253, '2 min', 30,
       76, 126, '7-8 minutes', 275, 69, 37, 43, 184, 250, 125, 56, 79,
       217, 114, 93, 38, 133, 64, 94, 124, 62], dtype=object)

There are some values below for which we can only approximate the seconds value logically

In [351]:
df_deduped.loc[df_deduped["duration"] == "Under two minutes", "duration"] = 119

In [352]:
df_deduped.loc[df_deduped["duration"] == "7-8 minutes", "duration"] = 450

In [353]:
df_deduped.loc[df_deduped["duration"] == "2 min", "duration"] = 120

In [354]:
df_deduped['duration_clean'] = df_deduped.duration.map(lambda x: convert_duration_to_seconds(str(x)))

In [355]:
df_deduped['duration_clean'].unique()

array([92, 120, 'nan', 180, 140, 167, 100, 190, 143, 130, 110, 135, 240,
       82, 90, 44, 128, 138, 160, 75, 96, 122, 148, 60, 105, 102, 108, 68,
       150, 156, 146, 52, 137, 170, 175, 145, 220, 84, 144, 51, 55, 115,
       171, 223, 116, 210, 83, 106, 73, 45, 66, 273, 200, 118, 85, 42,
       195, 139, 88, 112, 27, 63, 216, 77, 80, 67, 65, 300, 151, 99, 155,
       161, 213, 32, 356, 172, 154, 142, 70, 28, '119', 174, 104, 208,
       192, 40, 78, 153, 165, 127, 23, 36, 50, 103, 205, 87, 134, 117, 95,
       101, 113, 98, 136, '66', 25, 58, 253, '120', 30, 76, 126, '450',
       275, 69, 37, 43, 184, 250, 125, 56, 79, 217, 114, 93, 38, 133, 64,
       94, 124, 62], dtype=object)

In [356]:
df_deduped['duration_clean'] = df_deduped['duration_clean'].astype(float)

In [357]:
df_deduped.head()

,coaster_name,length,speed,location,status,opening_date,type,manufacturer,height_restriction,model,height,inversions,lift_launch_system,cost,trains,park_section,duration,capacity,g_force,designer,max_vertical_angle,drop,track_layout,year_introduced,latitude,longitude,type_main,length(m),speed(km/h),height_restriction_clean,height_clean,duration_clean
696,10 Inversion Roller Coaster,"2,788.8 ft (850.0 m)",45 mph (72 km/h),Chimelong Paradise,Operating,NaT,Steel,Intamin,54–77 in (137–196 cm),Looping Coaster (Colossus),98.4 ft (30.0 m),10.0,NaN,NaN,Single train with 7 cars. Riders are arranged ...,NaN,1:32,"1,300 riders per hour",NaN,Werner Stengel,NaN,NaN,NaN,2006,23.0044,113.3269,Steel,850.0,72.0,137.00,30.0,92.0
886,Abyss (roller coaster),"630 m (2,070 ft)",85 km/h (53 mph),Adventure World,Operating,2013-11-01,Steel,Gerstlauer,125 cm (4 ft 1 in),Euro-Fighter – Custom,30 m (98 ft),3.0,Chain lift hill,A$12 million,4 trains with a single car. Riders are arrange...,NaN,2 minutes,NaN,4.5,NaN,100°,30.5 m (100 ft),Custom,2013,-32.0955,115.8170,Steel,630.0,85.0,124.46,30.0,120.0
1057,Abyssus,"4,317.6 ft (1,316.0 m)",62.1 mph (99.9 km/h),Energylandia,Operating,2021-07-14,Steel,Vekoma,120 cm (3 ft 11 in),Shockwave 1320m+,126.3 ft (38.5 m),4.0,LSM,"€11,000,000",3 trains with 4 cars. Riders are arranged 2 ac...,Aqualantis,NaN,1000 riders per hour,4.5,Benjamin Bloemendaal,NaN,NaN,NaN,2021,49.9992,19.4011,Steel,1316.0,99.9,119.38,38.5,NaN
828,Accelerator (roller coaster),180 m (590 ft),60 km/h (37 mph),Drayton Manor,Operating,2011-04-21,Steel,Vekoma,100 cm (3 ft 3 in),Family Boomerang,20 m (66 ft),0.0,Booster Wheel Lift Hill,"£2,100,000",NaN,Aerial Park,NaN,750 riders per hour,3,Meticulous ltd,NaN,NaN,NaN,2011,52.6108,-1.7134,Steel,180.0,60.0,99.06,20.0,NaN
936,Acrobat (roller coaster),"3,349.8 ft (1,021.0 m)",55.9 mph (90.0 km/h),Nagashima Spa Land,Operating,2015-07-18,Steel,Bolliger & Mabillard,135–198 cm (4 ft 5 in–6 ft 6 in),Flying Coaster (Manta),141.1 ft (43.0 m),4.0,Chain lift hill,NaN,NaN,NaN,3:00,NaN,NaN,NaN,NaN,NaN,NaN,2015,NaN,NaN,Steel,1021.0,90.0,135.00,43.0,180.0


We will keep numerical columns as float type and other caategorical columns as string

In [358]:
df_deduped['length(m)'] = df_deduped['length(m)'].astype(float)

In [359]:
df_deduped['speed(km/h)'] = df_deduped['speed(km/h)'].astype(float)

In [360]:
df_deduped['status'] =df_deduped['status'].astype(str)

In [361]:
df_deduped['opening_date'] = df_deduped['opening_date'].astype("datetime64[ns]")

In [362]:
df_deduped['type'] = df_deduped['type'].astype(str)

In [363]:
df_deduped['height_restriction_clean'] = df_deduped['height_restriction_clean'].astype(float)

In [364]:
df_deduped['model'].unique()

array(['Looping Coaster (Colossus)', 'Euro-Fighter – Custom',
       'Shockwave 1320m+', 'Family Boomerang', 'Flying Coaster (Manta)',
       '320', 'Mine Train', 'Inverted Coaster – Custom', nan, 'Custom',
       'Inverted Coaster', 'Wooden', 'Custom Looping Coaster',
       'Hyper Coaster', 'Wild Mouse / Large Park',
       'Suspended looping coaster', 'Muti-loop', 'Bobsleigh', 'Dragon',
       'Junior Coaster (207m)', 'Wooden Coaster (Prefabricated Track)',
       'Cyclone', 'Sitdown', 'Dive Coaster',
       'Suspended Family Coaster (342m)', 'Inverted Coaster (Batman)',
       'Floorless Coaster[1]', 'Flying Dutchman (1018m)', 'Hypercoaster',
       'Sitdown Looping', 'Wooden Out and back coaster',
       'Floorless Coaster / Medusa',
       'Launched full-circuit dark ride / roller coaster',
       'Suspended Looping Coaster (765m Extended w/ Helix)',
       'Shuttle roller coaster', 'Motorbike Coaster (600m)',
       'Looping Coaster', 'Jumbo V',
       'Family Coasters/16ft Oval

In [365]:
df_deduped['height_clean'] = df_deduped['height_clean'].astype(float)

In [366]:
df_deduped['inversions'] = df_deduped['inversions'].astype(float)

In [367]:
df_deduped['lift_launch_system'].unique()

array([nan, 'Chain lift hill', 'LSM', 'Booster Wheel Lift Hill',
       'Two chain lift hills', 'Chain', 'Chain Lift', 'Powered',
       '1 chain lift hill', 'Chain lift', 'LSM launch', 'Chain Lift Hill',
       'Linear induction motor (LIM)[1]', 'Tire lift hill',
       'Two Chain lift hills', 'Elevator liftWheel lift', 'Chain-lift',
       'Three chain lift hills', 'Vertical chain lift hill',
       'Cable lift hill', 'LSM launch circuit coaster', '2 Chain Lifts',
       'Drive tire', 'LSM Launch', 'Drive tire lift hill', 'Spiral lift',
       'Vertical chain lift',
       'Trains are powered, propelling themselves on uphill sections. Trains then freely roll over drops. Tires embedded in the track also move trains through certain sections.',
       'Tire propelled launch, Chain lift hill',
       'Tire propelled lift hill', 'Powered coaster',
       'Hydraulic Launch Track', '3 linear synchronous motor launches',
       'Hydraulic Launch', 'Linear motor launch',
       'Linear synchr

Cleaning the cost column by removing unneccessary values in the data

In [368]:
df_deduped['cost'] = df_deduped['cost'].str.replace("\xa0", "").replace("[1]", "").replace("[2]", "").replace("[3]", "").str.strip()

In [369]:
df_deduped['cost'].unique()

array([nan, 'A$12 million', '€11,000,000', '£2,100,000', '$5,000,000',
       '$4,000,000', 'US$10.5 million', '$20 million', '€8,000,000',
       '$10 million', 'US$7,000,000', '$5,000,000[1]', '$10,000,000',
       'US$20 million', '$2.3 million', '€25 million', '£2,500,000',
       '100 million SEK (=approx. $12,6 million)', 'US$24,000,000',
       '€ 18 million[2]', '$9,500,000[2]', 'C$26 million est.',
       '$25 million', '£25,000 (1922)', 'USD15,000,000', '$2.25 million',
       '$200,000', '$3.044 million', 'US$6,000,000', 'US$60 million',
       '$22 million', '$2,000,000', '£13,500,000', '2.5 Million',
       '$175,000', '$26,000,000', '$1,750,000 USD', 'US$10 million',
       '$7,000,000', 'AU$30 million($30million in 2018 dollars[1])',
       '$9,000,000', '€4,000,000', '€ 20 million', 'CA$9 million[1]',
       '$7,000,000[1]', '75 million DKK (€10 M)',
       '3,500,000,000 Yen ($31,601,283 USD)', '€2,187,000',
       '$3,000,000 USD', 'US$100,000,000', 'US$12,100,000',
 

In [370]:
df_deduped['park_section'] = df_deduped['park_section'].str.replace("\xa0", "").str.strip()

In [371]:
df_deduped['park_section'] = df_deduped['park_section'].astype(str)

In [372]:
df_deduped['park_section'].unique()

array(['nan', 'Aqualantis', 'Aerial Park', 'South End', 'Oktoberfest',
       'Crossroads', 'Austria', 'Rhinefeld', 'County Fair',
       "1904 World's Fair", 'Jungle X-Pedition', 'The Underground',
       'Festa Italia', 'Candy Apple Grove',
       'ARTHUR - In the Minimoys Kingdom', 'Nickland', 'The Old West',
       'Action Zone', 'DC Super Heroes World', 'Gotham City',
       'Sci-Fi City', 'Frontier Adventures', 'Deep in Africa', 'Iceland',
       'Nickelodeon Land', 'Main Midway', 'Energy', 'Kiddie Land',
       'Africa', 'Pixar Pier', 'Chocolatetown', 'North Midway',
       'The Lost World', '2nd Level', 'Bugs Bunny World',
       'Carolina Boardwalk', 'DC Universe', 'Frontier Town',
       'Crown Colony Plaza', 'The Boardwalk', 'Egypt', 'Mexico',
       'Land der Vergessenen', 'Screampunk District', 'Lost City',
       'South Midway', 'The Hollow', 'Exotic', 'Blue Ridge Junction',
       'Gemini Midway', 'Planet Snoopy', 'Wuze Town',
       'Worlds of Pixar, Toon Studio', 'Nort

In [373]:
df_deduped['duration_clean'] = df_deduped.duration_clean.astype(float)

In [374]:
df_deduped.duration_clean.unique()

array([ 92., 120.,  nan, 180., 140., 167., 100., 190., 143., 130., 110.,
       135., 240.,  82.,  90.,  44., 128., 138., 160.,  75.,  96., 122.,
       148.,  60., 105., 102., 108.,  68., 150., 156., 146.,  52., 137.,
       170., 175., 145., 220.,  84., 144.,  51.,  55., 115., 171., 223.,
       116., 210.,  83., 106.,  73.,  45.,  66., 273., 200., 118.,  85.,
        42., 195., 139.,  88., 112.,  27.,  63., 216.,  77.,  80.,  67.,
        65., 300., 151.,  99., 155., 161., 213.,  32., 356., 172., 154.,
       142.,  70.,  28., 119., 174., 104., 208., 192.,  40.,  78., 153.,
       165., 127.,  23.,  36.,  50., 103., 205.,  87., 134., 117.,  95.,
       101., 113.,  98., 136.,  25.,  58., 253.,  30.,  76., 126., 450.,
       275.,  69.,  37.,  43., 184., 250., 125.,  56.,  79., 217., 114.,
        93.,  38., 133.,  64.,  94., 124.,  62.])

Cleaning the capacity_rider_per_hour the same way we had done before for other columns

In [375]:
df_deduped['capacity_rider_per_hr'] = df_deduped.capacity.str.replace(" riders per hour", "") \
                                        .str.replace(",", "") \
                                        .str.replace("[1]", "") \
                                        .str.replace("[2]", "") \
                                        .str.replace("[3]", "") \
                                        .str.replace("max.", "") \
                                        .str.replace("~", "") \
                                        .str.strip()

C:\Users\Dell\AppData\Local\Temp\ipykernel_17020\3614371917.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_deduped['capacity_rider_per_hr'] = df_deduped.capacity.str.replace(" riders per hour", "") \


In [376]:
df_deduped['capacity_rider_per_hr'].unique()

array(['00', nan, '000', '750', '600', '80', '800', '850', '400', '960',
       '780', '60', '650', '040', '545', '50', '500', '450 []', '760',
       '70', '950', '900', '440', '550', '050', '480-70', '0', '060',
       '008', '65', '758', '470', '700', '560', '45', '480', '640', '080',
       '5', '000[]', '746[]', '450', '44', '890', '670', '50[]', '4',
       '06', '680', '8', '596[]', '75', '90',
       '057 ( rockets) 74 (0 rockets) 440 (9 rockets)', '845', '460',
       '860', '400[]', '40', '975', 'cars - 8 riders each - 70',
       '700-800', '660', '905', '848', '886', '95', '85', '600[]', '9'],
      dtype=object)

In [377]:
df_deduped.loc[df_deduped['capacity_rider_per_hr'] == "480-720", "capacity_rider_per_hr"] = "600"

In [378]:
df_deduped.loc[df_deduped['capacity_rider_per_hr'] == "2057 (11 rockets) 1741 (10 rockets) 1440 (9 rockets)", "capacity_rider_per_hr"] = "2057"

In [379]:
df_deduped.loc[df_deduped['capacity_rider_per_hr'] == "3 cars - 8 riders each - 720", "capacity_rider_per_hr"] = "720"

In [380]:
df_deduped.loc[df_deduped['capacity_rider_per_hr'] == "700-800", "capacity_rider_per_hr"] = "750"

In [381]:
import pandas as pd

# Assuming df_deduped is your DataFrame

# Step 1: Clean the data
df_deduped['capacity_rider_per_hr'] = df_deduped['capacity_rider_per_hr'].str.replace('[]', '', regex=False)

# Step 2: Convert to numeric, forcing errors to NaN
df_deduped['capacity_rider_per_hr'] = pd.to_numeric(df_deduped['capacity_rider_per_hr'], errors='coerce')

# Now you can work with the column as float


In [382]:
df_deduped['g_force'].unique()

array([nan, '4.5', '3', '3.8', '3.7', '4.3', '5.1', '4.1', '2.2', '4.9',
       '4', '4G', '3.5', '2.5', '4.2', '1', '2.4', '5.2',
       '-1,5g and 4,5g', '4.9 (Back Seat)4.0 (Front Seat)',
       '4.5 G (Positive)', '3.75', '4.7', '0.2-2', '1.2', '4.4', '5',
       '2.9', '4.3 g', '3.6', '3.0', '3.2', '1.5', '2.1', '4.8[2]', '4.0',
       '3.26 G', '4[1]', '3.1', '3.4', '3.9', '4.18[4]', '4.8', '2.6',
       '3.5[2]', '3.7 [2]', 'Between 3 and 4 Gs', '2.75', '3.2[2]', '2.0',
       '3.8 g', '5.9', '2.3', '0.8', '3.3', '3 Gs[2]', '4.2 & -1', '4.6',
       '4.8[1]', '2.4[2]', '4 g', '3.5 g', '4Gs'], dtype=object)

Cleaning g_force column of the DataFrame

In [383]:
df_deduped['g_force_clean'] = df_deduped['g_force'] \
            .str.replace("[1]","") \
            .str.replace("[2]","") \
            .str.replace("[3]","") \
            .str.replace("[4]","") \
            .str.replace("Gs","") \
            .str.replace("g","")  \
            .str.replace("G","")  \
            .str.strip()

C:\Users\Dell\AppData\Local\Temp\ipykernel_17020\1679334538.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_deduped['g_force_clean'] = df_deduped['g_force'] \


## Cleaning ambiguous values in g_force_clean by using average or maximum values logically

In [392]:
df_deduped.loc[df_deduped['g_force_clean'] == '4.9 (Back Seat)4.0 (Front Seat)', 'g_force_clean'] = "4.5"
df_deduped.loc[df_deduped['g_force_clean'] == '4.5  (Positive)', 'g_force_clean'] = "4.5"
df_deduped.loc[df_deduped['g_force_clean'] == 'Between 3 and 4', 'g_force_clean'] = "3.5"
df_deduped.loc[df_deduped['g_force_clean'] == '-1,5 and 4,5', 'g_force_clean'] = "4.5"
df_deduped.loc[df_deduped['g_force_clean'] == '0.2-2', 'g_force_clean'] = "2"
df_deduped.loc[df_deduped['g_force_clean'] == '4.2 & -1', 'g_force_clean'] = "4"

# Step 1: Replace empty strings with NaN
df_deduped['g_force_clean'].replace('', pd.NA, inplace=True)

# Step 2: Convert to numeric, forcing errors to NaN
df_deduped['g_force_clean'] = pd.to_numeric(df_deduped['g_force_clean'], errors='coerce')

# Now you can check the result
print(df_deduped['g_force_clean'])


696     NaN
886     0.5
1057    0.5
828     NaN
936     NaN
       ... 
364     NaN
1046    NaN
433     NaN
56      NaN
434     5.0
Name: g_force_clean, Length: 646, dtype: float64


In [393]:
df_deduped['designer'].unique()

array(['Werner Stengel', nan, 'Benjamin Bloemendaal', 'Meticulous ltd',
       'Ing.-Büro Stengel GmbH',
       'Curtis D. SummersJames FigleyLeonard Wright', 'Ron Toomer',
       'Jeff Pike', 'Don Rosser / Bill Cobb',
       'Ingenieurbüro Stengel GmbH', 'Vekoma', 'TOGO', 'John A. Miller',
       'John C. Allen', 'Charlie Paige', 'Frank F. HooverJohn C. Allen',
       'The Gravity Group, LLC', 'Art Engineering[1]',
       'Dennis McNultyLarry Bill', 'Walt Disney Imagineering',
       'Dal Freeman (Lagoon)', 'Chance Rides', 'Zierer', 'Ride Tek',
       'Alan Schilke', 'Ingenieur Büro Stengel GmbH',
       'Herbert Paul Schmeck', 'Vernon Keenan', 'Edward A. Vettel',
       'Dennis McNulty, Larry Bill', 'The Gravity Group',
       'Karel Willemen', 'Frederick Church', 'HUSS Park Attractions',
       'John Wardley (layout) Ing.-Büro Stengel GmbH (mechanical role)',
       'Franz Mack', 'Reverchon Industries', 'Jack Rouse Associates',
       'Herschend Enterprises', 'Peter Clerx', 'Blauer 

Cleaning the designer column

In [394]:
df_deduped['designer_clean'] = df_deduped['designer'] \
    .str.replace("[1]", "") \
    .str.replace("[2]", "") \
    .str.replace("[3]", "") \
    .str.replace(r"\(?\d+\)?", "", regex=True) \
    .str.strip()
    

C:\Users\Dell\AppData\Local\Temp\ipykernel_17020\184381698.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_deduped['designer_clean'] = df_deduped['designer'] \


In [395]:
df_deduped['designer_clean'].unique()

array(['Werner Stengel', nan, 'Benjamin Bloemendaal', 'Meticulous ltd',
       'Ing.-Büro Stengel GmbH',
       'Curtis D. SummersJames FigleyLeonard Wright', 'Ron Toomer',
       'Jeff Pike', 'Don Rosser / Bill Cobb',
       'Ingenieurbüro Stengel GmbH', 'Vekoma', 'TOGO', 'John A. Miller',
       'John C. Allen', 'Charlie Paige', 'Frank F. HooverJohn C. Allen',
       'The Gravity Group, LLC', 'Art Engineering[]',
       'Dennis McNultyLarry Bill', 'Walt Disney Imagineering',
       'Dal Freeman (Lagoon)', 'Chance Rides', 'Zierer', 'Ride Tek',
       'Alan Schilke', 'Ingenieur Büro Stengel GmbH',
       'Herbert Paul Schmeck', 'Vernon Keenan', 'Edward A. Vettel',
       'Dennis McNulty, Larry Bill', 'The Gravity Group',
       'Karel Willemen', 'Frederick Church', 'HUSS Park Attractions',
       'John Wardley (layout) Ing.-Büro Stengel GmbH (mechanical role)',
       'Franz Mack', 'Reverchon Industries', 'Jack Rouse Associates',
       'Herschend Enterprises', 'Peter Clerx', 'Blauer E

In [396]:
df_deduped['max_vertical_angle'].unique()

array([nan, '100°', '97°', '79°', '55°', '65°', '45°', '70°', '87°',
       '33°', '75°', '46°', '59°', '77.3°', '116°', '53°', '61°', '80°',
       '60°', '47°', '58.1°', '120°', '50°', '78°', '52°', '95°', '74°',
       '90°', '87.5°', '89°', '82°', '81°', '51°',
       '40 degrees at bottom of first drop°', '3°', '85°', '64°',
       '120.5[4]°', '96°', '62°', '68°', '25°', '42°', '73°', '66°',
       '54°', '112°', '36°', '85[1]°', '27°', '35°', '29°', '77.4°',
       '53.3°', '40°', '68.5°', '56°', '111°', '72°', '58°', '77°',
       '121.5°', '121°', '101°', '57.3°', '113.1°', '83°', '63°', '88.5°'],
      dtype=object)

### We dont want "°" in the max_vetical_angle column as it is understood that these values represent angle so these values can be removed for easy searching later

In [400]:
# Step 1: Clean the data
df_deduped['max_vertical_angle_clean'] = (
    df_deduped['max_vertical_angle']
    .str.replace("°", "", regex=False)
    .str.replace("[1]", "", regex=False)
    .str.replace("[4]", "", regex=False)
)

# Step 2: Replace specific string cases
df_deduped.loc[df_deduped['max_vertical_angle_clean'] == '40 degrees at bottom of first drop', 'max_vertical_angle_clean'] = "40"

# Step 3: Extract numeric values using regex
df_deduped['max_vertical_angle_clean'] = df_deduped['max_vertical_angle_clean'].str.extract('(\d+)')

# Step 4: Convert to numeric, forcing errors to NaN
df_deduped['max_vertical_angle_clean'] = pd.to_numeric(df_deduped['max_vertical_angle_clean'], errors='coerce')

# Now you can check the result
print(df_deduped['max_vertical_angle_clean'])


696       NaN
886     100.0
1057      NaN
828       NaN
936       NaN
        ...  
364      40.0
1046     90.0
433       NaN
56        NaN
434       NaN
Name: max_vertical_angle_clean, Length: 646, dtype: float64


In [401]:
df_deduped['max_vertical_angle_clean'].unique()

array([ nan, 100.,  97.,  79.,  55.,  65.,  45.,  70.,  87.,  33.,  75.,
        46.,  59.,  77., 116.,  53.,  61.,  80.,  60.,  47.,  58., 120.,
        50.,  78.,  52.,  95.,  74.,  90.,  89.,  82.,  81.,  51.,  40.,
         3.,  85.,  64.,  96.,  62.,  68.,  25.,  42.,  73.,  66.,  54.,
       112.,  36.,  27.,  35.,  29.,  56., 111.,  72., 121., 101.,  57.,
       113.,  83.,  63.,  88.])

In [402]:
df_deduped['drop'].unique()

array([nan, '30.5\xa0m (100\xa0ft)', '70\xa0ft (21\xa0m)',
       '170\xa0ft (52\xa0m)', '147\xa0ft (44.8\xa0m)',
       '80\xa0ft (24\xa0m)', '144\xa0ft (44\xa0m)',
       '87.3\xa0ft (26.6\xa0m)', '210\xa0ft (64\xa0m)',
       '50\xa0ft (15\xa0m)', '92\xa0ft (28\xa0m)', '24.9\xa0m (82\xa0ft)',
       '150\xa0ft (46\xa0m)', '37.5\xa0m (123\xa0ft)',
       '103\xa0ft (31\xa0m)', '69\xa0m (226\xa0ft)',
       '132\xa0ft (40\xa0m)', '72\xa0ft (22\xa0m)',
       '98.43\xa0ft (30.00\xa0m)', '115\xa0ft (35\xa0m)',
       '108\xa0ft (33\xa0m)', '66\xa0ft (20\xa0m)', '6\xa0ft (1.8\xa0m)',
       '65\xa0ft (20\xa0m)', '34\xa0ft (10\xa0m)', '20\xa0ft (6.1\xa0m)',
       '130\xa0ft (40\xa0m)', '48.5\xa0m (159\xa0ft)',
       '128\xa0ft (39\xa0m)', '97\xa0ft (30\xa0m)', '81\xa0ft (25\xa0m)',
       '78\xa0ft (24\xa0m)', '25\xa0ft (7.6\xa0m)', '30\xa0m (98\xa0ft)',
       '100\xa0ft (30\xa0m)', '75\xa0ft (23\xa0m)',
       '59.4\xa0m (195\xa0ft)', '215\xa0ft (66\xa0m)',
       '179\xa0ft (55\xa0m)

Cleaning the "\xa" values from the drop column

In [403]:
df_deduped['drop'].str.replace("\xa0", "").unique()

array([nan, '30.5m (100ft)', '70ft (21m)', '170ft (52m)', '147ft (44.8m)',
       '80ft (24m)', '144ft (44m)', '87.3ft (26.6m)', '210ft (64m)',
       '50ft (15m)', '92ft (28m)', '24.9m (82ft)', '150ft (46m)',
       '37.5m (123ft)', '103ft (31m)', '69m (226ft)', '132ft (40m)',
       '72ft (22m)', '98.43ft (30.00m)', '115ft (35m)', '108ft (33m)',
       '66ft (20m)', '6ft (1.8m)', '65ft (20m)', '34ft (10m)',
       '20ft (6.1m)', '130ft (40m)', '48.5m (159ft)', '128ft (39m)',
       '97ft (30m)', '81ft (25m)', '78ft (24m)', '25ft (7.6m)',
       '30m (98ft)', '100ft (30m)', '75ft (23m)', '59.4m (195ft)',
       '215ft (66m)', '179ft (55m)', '23m (75ft)', '23.2m (76ft)',
       '49.1m (161ft)', '20m (66ft)', '21.5m (71ft)', '135ft (41m)',
       '82ft (25m)', '105ft (32m)', '168ft (51m)', '35ft (11m)',
       '30ft (9.1m)', '91ft (28m)', '13ft (4.0m)', '230ft (70m)',
       '41.9m (137ft)', '320ft (98m)', '164ft (50m)', '118ft (36.0m)',
       '101ft (31m)', '12m (39ft)', '60ft (18m)',

### Now Standardising the drop column units to metres

In [404]:
df_deduped['drop_clean'] = df_deduped['drop'].str.extract(r'([\d,\.]+)\s*m').replace(',', '', regex=True)[0]

In [405]:
df_deduped['drop_clean'] = df_deduped['drop_clean'].astype(float)

Ensuring the other columns have proper values by checking out all of the unique values of the column

In [406]:
df_deduped['track_layout'].unique()

array([nan, 'Custom', 'Terrain', 'Out and back', 'Out and Back/Terrain',
       'Wild Mouse', 'Loop', 'Inverted', 'Twister', 'Out and Back',
       'Steel', 'Single Rail', 'Double figure 8[1]',
       'Möbius Loop with one station', 'Intamin Tri Track', 'Figure 8',
       'Modified Double Out and Back', 'Wood', 'Compact Twister',
       'Water Coaster', 'Out & back', 'Figure Eight', 'Indoor',
       'Double Out and Back',
       '8 layers laminated wood strips with 1/4"x3" wide steel rail',
       'Out-and-back', 'Single Helix (center)',
       'Custom - Infinity Coaster', 'Double Figure Eight',
       'Out and Back Helix and Twister', 'Pinfari TL-59',
       'Terrain roller coaster', 'Terrain, Out and Back roller coaster',
       'Terrain, Out and Back', 'Wooden out-and-back', 'Oval',
       'Indoor-Spinning Glow', 'Dual-tracked', 'Shuttle Loop',
       'Stacked Figure-8', 'L-shaped Out and Back',
       'Terrain Mega Coaster', 'Terrain Wood Twister Out and Back',
       'Möbius Loop'

In [407]:
df_deduped['year_introduced'].unique()

array([2006, 2013, 2021, 2011, 2015, 2018, 1991, 1999, 2010, 1984, 1997,
       2014, 1981, 2008, 1989, 2009, 2002, 1978, 2003, 1988, 1995, 1996,
       2005, 2000, 2001, 1923, 1958, 1964, 1998, 2007, 2004, 2020, 1967,
       1993, 1980, 1973, 1969, 2016, 1983, 1946, 1951, 1994, 1927, 2019,
       1976, 1979, 1940, 2017, 2012, 1992, 1929, 1987, 1972, 1924, 1925,
       1971, 1982, 1920, 1985, 1902, 1950, 1952, 1959, 1928, 1933, 1990,
       1910, 1975, 1932, 1921, 1938, 1974, 1966, 1956, 1977, 1912, 1986,
       1968, 1941, 1930, 1934], dtype=int64)

In [408]:
df_deduped['latitude'].unique()

array([ 23.0044, -32.0955,  49.9992,  52.6108,      nan,  45.4716,
        39.3445,  35.1003,  28.0379,  37.2328,  48.2617,  42.3646,
        38.5125,  37.8392,  34.4213,  37.2348,  34.5213,  48.2639,
        53.7901,  44.8542,  51.6198,  57.6933,  51.6242,  35.6255,
        39.3457,  51.6482,  40.9862,  40.2282,  42.0353,  19.2952,
         1.2541,  38.9103,  43.839 ,  36.1019,  53.7892,  40.1382,
        40.8797,  50.7986,  48.2624,  53.7902,  41.4799,  45.4566,
        29.5463,  40.9855,  39.1731,  51.3975,  41.6392,  33.8046,
        40.2863,  40.9874,  52.5062,   1.2534,  36.1381,  34.4249,
        35.1042,  42.0371,  41.4847,  28.0342,  33.845 ,  63.2298,
        56.1358,  50.7987,  53.0231,  34.4278,  51.4032,  40.9836,
        40.2864,  42.1077,  52.4404,  40.5742,  35.1011,  41.4836,
        43.3474,  44.7996,  40.7898,  39.1723,  50.801 ,  39.1847,
        48.8682,  39.866 ,  39.7808,  42.0403,  43.6161, -27.908 ,
        33.7676,  33.7703,  30.5912,  50.8483,  51.648 ,  18.7

In [409]:
df_deduped['longitude'].unique()

array([ 113.3269,  115.817 ,   19.4011,   -1.7134,       nan, -122.6623,
        -84.2648,  -80.9409,  -82.4216,  -76.6476,    4.6149,  -87.9336,
        -90.6772,  -77.4402, -118.5998,  -76.6413,  -93.0143,    7.7237,
         -3.0559,  -93.2422,    6.976 ,   11.9961,    6.9754,  139.5181,
        -84.2655,    5.0508, -111.8926,   -3.5924,  -72.6131,  -99.2065,
        103.8223,  -76.7786,  -79.5406, -115.1736,   -3.0569,  -74.4323,
        -76.5068,    6.8803,    7.7185,   -3.0539,  -82.6826,   10.7137,
        -95.0173, -111.8923,  -94.4851,    5.9842,  -72.9245, -117.9218,
        -76.659 , -111.8955,    1.7451,  103.824 , -115.1664, -118.5964,
        -80.9436,  -72.6117,  -82.6904,  -82.42  , -118.0019,   22.8587,
         10.1979,    6.8815,    9.8833, -118.5975,   -0.513 , -111.8951,
        -76.6556,  -80.1568,    5.7615,  -73.9778,  -80.9426,  -82.6853,
        -86.2789,  -93.4563,  -86.771 ,  -94.4889,    6.878 ,  117.4686,
          2.7777,  116.4885, -105.054 ,  -72.6156, 

In [410]:
df_deduped

,coaster_name,length,speed,location,status,opening_date,type,manufacturer,height_restriction,model,height,inversions,lift_launch_system,cost,trains,park_section,duration,capacity,g_force,designer,max_vertical_angle,drop,track_layout,year_introduced,latitude,longitude,type_main,length(m),speed(km/h),height_restriction_clean,height_clean,duration_clean,capacity_rider_per_hr,g_force_clean,designer_clean,max_vertical_angle_clean,drop_clean
696,10 Inversion Roller Coaster,"2,788.8 ft (850.0 m)",45 mph (72 km/h),Chimelong Paradise,Operating,NaT,Steel,Intamin,54–77 in (137–196 cm),Looping Coaster (Colossus),98.4 ft (30.0 m),10.0,NaN,NaN,Single train with 7 cars. Riders are arranged ...,nan,1:32,"1,300 riders per hour",NaN,Werner Stengel,NaN,NaN,NaN,2006,23.0044,113.3269,Steel,850.0,72.0,137.00,30.0,92.0,0.0,NaN,Werner Stengel,NaN,NaN
886,Abyss (roller coaster),"630 m (2,070 ft)",85 km/h (53 mph),Adventure World,Operating,2013-11-01,Steel,Gerstlauer,125 cm (4 ft 1 in),Euro-Fighter – Custom,30 m (98 ft),3.0,Chain lift hill,A$12 million,4 trains with a single car. Riders are arrange...,nan,2 minutes,NaN,4.5,NaN,100°,30.5 m (100 ft),Custom,2013,-32.0955,115.8170,Steel,630.0,85.0,124.46,30.0,120.0,NaN,0.5,NaN,100.0,30.5
1057,Abyssus,"4,317.6 ft (1,316.0 m)",62.1 mph (99.9 km/h),Energylandia,Operating,2021-07-14,Steel,Vekoma,120 cm (3 ft 11 in),Shockwave 1320m+,126.3 ft (38.5 m),4.0,LSM,"€11,000,000",3 trains with 4 cars. Riders are arranged 2 ac...,Aqualantis,NaN,1000 riders per hour,4.5,Benjamin Bloemendaal,NaN,NaN,NaN,2021,49.9992,19.4011,Steel,1316.0,99.9,119.38,38.5,NaN,0.0,0.5,Benjamin Bloemendaal,NaN,NaN
828,Accelerator (roller coaster),180 m (590 ft),60 km/h (37 mph),Drayton Manor,Operating,2011-04-21,Steel,Vekoma,100 cm (3 ft 3 in),Family Boomerang,20 m (66 ft),0.0,Booster Wheel Lift Hill,"£2,100,000",NaN,Aerial Park,NaN,750 riders per hour,3,Meticulous ltd,NaN,NaN,NaN,2011,52.6108,-1.7134,Steel,180.0,60.0,99.06,20.0,NaN,750.0,NaN,Meticulous ltd,NaN,NaN
936,Acrobat (roller coaster),"3,349.8 ft (1,021.0 m)",55.9 mph (90.0 km/h),Nagashima Spa Land,Operating,2015-07-18,Steel,Bolliger & Mabillard,135–198 cm (4 ft 5 in–6 ft 6 in),Flying Coaster (Manta),141.1 ft (43.0 m),4.0,Chain lift hill,NaN,NaN,nan,3:00,NaN,NaN,NaN,NaN,NaN,NaN,2015,NaN,NaN,Steel,1021.0,90.0,135.00,43.0,180.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,Zach's Zoomer,NaN,NaN,Michigan's Adventure,Operating,1994-07-22,Wood,Custom Coasters International,40 in (102 cm),NaN,NaN,0.0,Chain Lift Hill,NaN,Single train with 5 cars. Riders are arranged ...,nan,1:20,450 riders per hour,NaN,NaN,40°,NaN,NaN,1994,43.3434,-86.2809,Wood,NaN,NaN,102.00,NaN,80.0,450.0,NaN,NaN,40.0,NaN
1046,Zadra (roller coaster),"1,316 m (4,318 ft)",121 km/h (75 mph),Energylandia,Operating,2019-08-22,Other,Rocky Mountain Construction,140 cm (4 ft 7 in),NaN,62.8 m (206 ft),3.0,Chain lift hill,NaN,2 trains with 6 cars. Riders are arranged 2 ac...,Dragon Zone,1:50,"1,050 riders per hour",4,NaN,90°,NaN,NaN,2019,50.0019,19.4031,Other,1316.0,121.0,139.70,62.8,110.0,50.0,NaN,NaN,90.0,NaN
433,Zeus (roller coaster),"2,900 ft (880 m)",60 mph (97 km/h),Mt. Olympus Water & Theme Park,Operating,1997-06-07,Wood,Custom Coasters International,48 in (122 cm),NaN,90 ft (27 m),NaN,NaN,NaN,NaN,nan,1:25,NaN,NaN,NaN,NaN,85 ft (26 m),NaN,1997,43.6167,-89.7892,Wood,880.0,97.0,122.00,27.0,85.0,NaN,NaN,NaN,NaN,26.0
56,Zipper Dipper,NaN,NaN,Blackpool Pleasure Beach,Operating,NaT,Other,NaN,NaN,NaN,15 ft (4.6 m),0.0,NaN,NaN,Single train with 5 cars. Riders are arranged ...,Nickelodeon Land,1:00,NaN,NaN,Charlie Paige,NaN,NaN,NaN,1934,53.7902,-3.0539,Other,NaN,NaN,NaN,4.6,60.0,NaN,NaN,Charlie Paige,NaN,NaN


`Converting the data to a csv file`

In [411]:
df_deduped.to_csv("enrichedRollerCoaster.csv")

-------------------

# Thank you! 😊🎉
